In [7]:
import pandas as pd
from bertopic import BERTopic
from umap import UMAP
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import HDBSCAN
from stopwordsiso import stopwords

import warnings
warnings.simplefilter("ignore")

# Glassdoor

In [3]:
path = "../data/glassdoor_reviews_clean.csv"
df = pd.read_csv(path, on_bad_lines="skip")
pros = df[['review_pros', 'len_review_pros']].dropna().query('len_review_pros <= 40').sample(50000, random_state=123)
cons = df[['review_cons', 'len_review_cons']].dropna().query('len_review_cons <= 40').sample(50000, random_state=123)

In [6]:
pros_docs = pros['review_pros'].tolist()
cons_docs = cons['review_cons'].tolist()

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
pros_embeddings = embedding_model.encode(pros_docs, show_progress_bar=True, convert_to_numpy=True)
cons_embeddings = embedding_model.encode(cons_docs, show_progress_bar=True, convert_to_numpy=True)

Batches: 100%|██████████| 1563/1563 [00:06<00:00, 236.31it/s]


## Pros

In [ ]:
pros_model = BERTopic(verbose=True,
                      min_topic_size=100, 
                      calculate_probabilities=True,
                      embedding_model=embedding_model,
                      vectorizer_model=CountVectorizer(stop_words=list(stopwords('en'))),
                      umap_model=UMAP(random_state=123)),

topics_p, probs_p = pros_model.fit_transform(pros_docs, pros_embeddings)

topics_p = pros_model.reduce_outliers(pros_docs, topics=topics_p, probabilities=probs_p, strategy="embeddings")
pros_model.update_topics(pros_docs, topics=topics_p)

In [36]:
for row in pros_model.get_topic_info().iterrows():
    print(f"Topic: {row[1]['Name']}, N= {row[1]['Count']}")

Topic: 0_culture_work_great_and, N= 2468
Topic: 1_flexible_schedule_hours_scheduling, N= 2544
Topic: 2_team_teams_great_teamwork, N= 2008
Topic: 3_environment_atmosphere_friendly_fun, N= 2060
Topic: 4_free_food_lunch_coffee, N= 1597
Topic: 5_balance_life_work_good, N= 1629
Topic: 6_place_work_to_great, N= 1833
Topic: 7_coworkers_workers_co_nice, N= 1639
Topic: 8_pay_decent_salary_good, N= 1924
Topic: 9_students_campus_teachers_faculty, N= 994
Topic: 10_none_nothing_think_this, N= 787
Topic: 11_discount_discounts_employee_dress, N= 823
Topic: 12_pros_pro_no_there, N= 669
Topic: 13_technology_projects_technologies_tech, N= 1107
Topic: 14_patients_patient_care_hospital, N= 681
Topic: 15_management_managers_manager_is, N= 1123
Topic: 16_easy_job_you_it, N= 976
Topic: 17_home_from_work_working, N= 585
Topic: 18_staff_friendly_colleagues_very, N= 751
Topic: 19_company_for_great_to, N= 1926
Topic: 20_customers_customer_service_store, N= 672
Topic: 21_pto_unlimited_benefits_generous, N= 432
To

## Cons

In [ ]:
cons_model = BERTopic(verbose=True,
                      min_topic_size=170, 
                      calculate_probabilities=True,
                      embedding_model=embedding_model)

topics_c, probs_c = cons_model.fit_transform(cons_docs, cons_embeddings)

topics_c = cons_model.reduce_outliers(cons_docs, topics=topics_c, probabilities=probs_c, strategy="embeddings")
cons_model.update_topics(cons_docs, topics=topics_c)

In [41]:
for row in cons_model.get_topic_info().iterrows():
    print(f"Topic: {row[1]['Name']}, N= {row[1]['Count']}")

Topic: 0_pay_low_salary_is, N= 5655
Topic: 1_none_nothing_say_think, N= 4150
Topic: 2_work_can_stressful_be, N= 3186
Topic: 3_cons_any_no_there, N= 1678
Topic: 4_management_managers_employees_and, N= 2880
Topic: 5_balance_life_work_no, N= 1267
Topic: 6_hours_long_time_schedule, N= 2388
Topic: 7_cons_con_no_this, N= 1183
Topic: 8_customers_rude_customer_store, N= 1648
Topic: 9_company_the_growing_to, N= 2263
Topic: 10_benefits_insurance_health_no, N= 1202
Topic: 11_training_to_you_learn, N= 1054
Topic: 12_technology_processes_systems_slow, N= 1177
Topic: 13_career_advancement_opportunities_growth, N= 1219
Topic: 14_poor_management_lack_pay, N= 2088
Topic: 15_shifts_shift_night_hours, N= 679
Topic: 16_promotion_promotions_promoted_get, N= 688
Topic: 17_students_school_teachers_the, N= 690
Topic: 18_leadership_senior_leaders_lack, N= 1007
Topic: 19_culture_and_diversity_the, N= 632
Topic: 20_patient_patients_hospital_care, N= 600
Topic: 21_politics_bureaucracy_political_too, N= 986
Topic:

# Indeed Reviews

In [5]:
path_indeed = "../data/indeed_reviews_clean.csv"
df_indeed = pd.read_csv(path_indeed, on_bad_lines="skip")
reviews_indeed = df_indeed[['main_review', 'len_main_review']].dropna().query('len_main_review <= 150').sample(100000, random_state=123)

In [8]:
docs_indeed = reviews_indeed['main_review'].tolist()
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
embeddings_indeed = embedding_model.encode(docs_indeed, show_progress_bar=True, convert_to_numpy=True)

Batches: 100%|██████████| 3125/3125 [00:27<00:00, 112.31it/s]


In [16]:
indeed_model = BERTopic(verbose=True,
                        calculate_probabilities=False,
                        embedding_model=embedding_model,
                        vectorizer_model=CountVectorizer(stop_words=list(stopwords('en'))),
                        umap_model=UMAP(random_state=123, n_jobs=16), 
                        hdbscan_model = HDBSCAN(min_cluster_size=90, n_jobs=16))

topics_indeed, probs_indeed = indeed_model.fit_transform(docs_indeed, embeddings_indeed)

topics_indeed = indeed_model.reduce_outliers(docs_indeed, topics=topics_indeed, probabilities=probs_indeed, strategy="embeddings")
indeed_model.update_topics(docs_indeed, topics=topics_indeed)

2026-02-10 13:35:52,065 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-10 13:37:23,316 - BERTopic - Dimensionality - Completed ✓
2026-02-10 13:37:23,318 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-10 13:38:16,896 - BERTopic - Cluster - Completed ✓
2026-02-10 13:38:16,911 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-02-10 13:38:18,866 - BERTopic - Representation - Completed ✓
2026-02-10 13:38:25,231 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [17]:
for row in indeed_model.get_topic_info().iterrows():
    print(f"Topic: {row[1]['Name']}, N= {row[1]['Count']}")

Topic: 0_to_and_the_work, N= 57708
Topic: 1_army_military_the_in, N= 5244
Topic: 2_patients_patient_care_hospital, N= 4792
Topic: 3_what_company_at_like, N= 1780
Topic: 4_students_children_kids_and, N= 2348
Topic: 5_guard_national_army_the, N= 981
Topic: 6_food_restaurant_the_and, N= 1935
Topic: 7_air_force_usaf_the, N= 901
Topic: 8_navy_the_in_of, N= 852
Topic: 9_walmart_store_mart_wal, N= 857
Topic: 10_call_calls_center_phone, N= 1174
Topic: 11_drivers_driver_trucks_truck, N= 1329
Topic: 12_mcdonald_mcdonalds_burger_king, N= 572
Topic: 13_bank_banking_financial_and, N= 859
Topic: 14_de_en_que_el, N= 398
Topic: 15_pharmacy_pharmacist_prescriptions_and, N= 508
Topic: 16_marine_corps_marines_the, N= 434
Topic: 17_ups_you_at_for, N= 379
Topic: 18_toyota_nissan_honda_nordstrom, N= 426
Topic: 19_travel_world_and_people, N= 1369
Topic: 20_disney_walt_cast_guests, N= 327
Topic: 21_country_serve_serving_my, N= 615
Topic: 22_verizon_wireless_and_to, N= 359
Topic: 23_at_company_att_to, N= 323
T